# Bay Area Airbnb Review Classification
### Capstone Part 4: Classification Analysis and Visualization
Lauren Mitchell 2022

#### Project Objective 
The objective of this project is to take Airbnb reviews from available web datasets and perform Natural Language Processing on the text of the reviews to evaluate them on sentiment analysis and then cluster them based on possible issues with the Airbnbs. 

#### Notebook Objective
This notebook tackles the fourth and final step which includes performing an EDA on the output categories to gain meaningful insights and building visualizations.

Notebook Outline

Section 1: Final Data Processing
1. Data Import
2. Data Processing 
3. Get Dummies
4. Make Final Dataframes for Visualization


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('display.max_colwidth', None)

# Section 1: Final Data Processing 
### Data Import

In [2]:
from google.colab import files
uploaded = files.upload()

Saving final_review_categories.csv to final_review_categories.csv
Saving listings_and_reviews.csv to listings_and_reviews.csv
Saving df_negative_sentiment.csv to df_negative_sentiment.csv


In [3]:
# importing the csv files into dataframes
final_categories = pd.read_csv('final_review_categories.csv', sep='\t')
listings_and_reviews = pd.read_csv('listings_and_reviews.csv', sep='\t')
df_negative_sentiment = pd.read_csv('df_negative_sentiment.csv', sep='\t')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,3,7,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# Processing Final DataFrame

#### Making sure final_categories and df_negative sentiment both have the linking column unique_review_code

In [9]:
final_categories.head(3)

,Unnamed: 0,unique_review_code,sentence,category
0,0,33437,"If you are a light sleeper or are really bothered by the noise, I wouldn’t recommend.",8
1,1,11914,"Overall, a very strange and shady situation.",4
2,2,73115,My family was there to help my cousin move into Berkeley.,0


In [10]:
df_negative_sentiment.sample(1)

,Unnamed: 0,index,listing_id,date,reviewer_id,reviewer_name,comments,name,host_id,host_name,...,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,unique_review_code,neg,pos,pos_minus_neg,index_key
5945,5945,5945,50075123,2021-12-31,71312660,Maxine,"Had a great stay, host was immaculate. Only trouble was parking.",14 mins to sf! room with private bath and backyard,70559334,Valerie,...,2022-03-18,8.79,1,360,85,75107,0.171,0.259,0.088,5945


### Creating dummy variables for category, so we can group them by review. 

Recall that most reviews have more than one sentence so they could have more than one category.

In [11]:
dummy_categories = pd.get_dummies(final_categories['category'], prefix= 'cat', drop_first = True)
dummy_categories.shape

(27918, 10)

In [12]:
dummy_categories

,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10
0,0,0,0,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
27913,0,0,0,0,0,0,0,0,0,0
27914,0,0,0,0,0,0,0,0,0,0
27915,0,0,0,0,0,0,0,0,0,0
27916,0,1,0,0,0,0,0,0,0,0


In [13]:
final_labeled_with_dummies = pd.concat([final_categories, dummy_categories], axis = 1)
print(final_labeled_with_dummies.shape)
final_labeled_with_dummies.head()

(27918, 14)


,Unnamed: 0,unique_review_code,sentence,category,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10
0,0,33437,"If you are a light sleeper or are really bothered by the noise, I wouldn’t recommend.",8,0,0,0,0,0,0,0,1,0,0
1,1,11914,"Overall, a very strange and shady situation.",4,0,0,0,1,0,0,0,0,0,0
2,2,73115,My family was there to help my cousin move into Berkeley.,0,0,0,0,0,0,0,0,0,0,0
3,3,70385,"We were only here for a few days while we moved out of state, so we did not do any exploring around the area.",0,0,0,0,0,0,0,0,0,0,0
4,4,53402,Sierra siempre muy amable y servicial.,0,0,0,0,0,0,0,0,0,0,0


### Grouping by the categories and summing so we get a total for each review. 

In [16]:
complaints_grouped = final_labeled_with_dummies.groupby('unique_review_code').agg(
             cat_1 = ('cat_1', 'sum'),
             cat_2 = ('cat_2', 'sum'),
             cat_3 = ('cat_3', 'sum'),
             cat_4 = ('cat_4', 'sum'),
             cat_5 = ('cat_5', 'sum'),
             cat_6 = ('cat_1', 'sum'),
             cat_7 = ('cat_1', 'sum'),
             cat_8 = ('cat_1', 'sum'),
             cat_9 = ('cat_1', 'sum'),
             cat_10 = ('cat_1', 'sum'))

complaints_grouped.reset_index()  

,unique_review_code,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10
0,11,0,0,0,1,0,0,0,0,0,0
1,22,0,0,0,0,0,0,0,0,0,0
2,59,0,0,0,0,0,0,0,0,0,0
3,73,0,0,2,0,0,0,0,0,0,0
4,85,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6120,77531,0,0,0,0,0,0,0,0,0,0
6121,77535,1,0,0,0,0,1,1,1,1,1
6122,77538,0,0,0,0,0,0,0,0,0,0
6123,77572,0,2,4,0,0,0,0,0,0,0


# Merging the Dataframe with the categories with the dataframe with the reviews

In [19]:
print(df_negative_sentiment.shape)
print(complaints_grouped.shape)

(6125, 27)
(6125, 10)


In [24]:
negative_reviews_summary = pd.merge(df_negative_sentiment, complaints_grouped, on = 'unique_review_code', how="inner")

In [29]:
negative_reviews_summary = negative_reviews_summary.drop(columns=['Unnamed: 0','index'])

In [30]:
negative_reviews_summary.T

,0,1,2,3,4,5,6,7,8,9,...,6115,6116,6117,6118,6119,6120,6121,6122,6123,6124
listing_id,3083,3083,5739,5739,5739,5739,5739,5739,5739,23637,...,54263926,54282494,540688083947295343,544473512721001380,548799061903797260,549265183775803016,550870638201389270,551425151303360861,571848052601224459,574751225069531168
date,2018-10-15,2021-03-01,2018-07-28,2018-12-14,2019-04-28,2019-07-27,2019-12-15,2019-12-29,2020-07-03,2018-05-04,...,2022-03-07,2022-01-19,2022-03-07,2022-03-11,2022-02-11,2022-03-06,2022-03-07,2022-02-13,2022-03-11,2022-03-06
reviewer_id,63285107,386948436,107547829,188730681,71802380,233825357,68074399,216481080,1609664,84749793,...,84379524,422664217,397715913,133613400,3656690,365748042,363786852,154918813,362955706,217596381
reviewer_name,Alberto,Nicholas,若文,Emily,Brett,Sharon,Ian,Kenny,Chris,Janet,...,Terry,Tracy,Tequila,Alejandro,Nate,Greta,Jamell,Iffatara,Deandrea,Melissa
comments,"Clean, but outdated place with cheap furniture falling apart. It is located in a sketchy area 4 blocks away from the Barc metro station. If You are a light sleeper, that’s no a place to be. You will be hearing the neighbors (starting at 6 am), the steps on top your bedroom and at 4 am maybe a dog playing with a toy on top your bedroom (vs. haunted activities)",Homely feel,非常好的房东，装修很用心。一切都很完美。位置也很不错，距离bart不远。Perfect,Leah and Gyorgy's place is in a great location close to Lake Merritt. The house is charming and there is a little patio out front though I didn't get a chance to use it during my stay. The basement studio is spacious and includes a desk and living area with couch. It was cold during my stay but they provided a space heater and robes which was a nice touch. The stay was very personal with Gyorgy letting me into the studio at check-in and checking in throughout the stay to see if I needed anything.,"Leah and Gyorgy are possibly the best hosts I’ve ever had on Airbnb. The Mod Studio is not only easily accessible, but is comfortable, stylish and made me feel right at home. I can not recommend their place enough!","The Mod Studio is the place to be when staying in Oakland, so much to do, great location and minutes away from Lake Merritt which is an oasis walking around the park. Close to transportation, and the studio is all you need. You will not be disappointed when I return again this is where I'll stay hands down. Leah and Gyorgy are so accommodating whatever you need they got you.",Very stylish studio within a short walk of Downtown Oakland.,"Very difficult to imagine a better place to stay or finer hosts. Photographs of the Mod Studio do convey some of its charm, but cannot give an idea of just how many amenities are compactly and conveniently provided. The location could not be better, and the comfort is complete. Highest rating possible.","I'm an Oakland resident who needed a place to stay while my apartment was getting fixed and this was a nice break in a different neighborhood. I appreciated that during the pandemic the hosts took cleanliness and disease risks seriously. This is a cool ""garden"" apartment in the true sense of the word: your windows look into a garden and you have your entrance from the garden. Bed is a ""full"" size made up of mattresses of two different kinds of hardness on the floor; I found it comfortable. There's also a couch and desk and wifi is strong. I didn't try the TV. One of the hosts is a music teacher so you may get some pleasant guitar sounds during his lessons if it's during the summer. It's in the Adams Point neighborhood so a Whole Foods is available for provisions and they provide a large dorm fridge for keeping snacks. Once the pandemic eases, you'll also find the neighborhood has several restaurants and coffee places to choose from. Only downside is probably transportation, as parking is difficult in the neighborhood and BART is probably a 20-min walk.",The host canceled this reservation 10 days before arrival. This is an automated posting.,...,Very spacious and beautiful place. Ju

# Merging with the listings and reviews dataframe

In [34]:
print(listings_and_reviews.shape)
listings_and_reviews.head(1)

(77592, 25)


,Unnamed: 0,listing_id,date,reviewer_id,reviewer_name,comments,name,host_id,host_name,neighbourhood,...,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,unique_review_code,neg,pos,pos_minus_neg
0,0,3083,2018-03-25,37009535,Johannes,"Realy good opportunity for a cheap stay in San Francisco close to the City Center (approx. 20 min to Powel Street station). Guesthouse is nice and cosy and has an amazing garden. The neighbourhood might feel a little scary after dark, but to be fair I can not really compare it to other places in San Francisco and we never had any problems.",Room #2 @ B-Love's Guest House,3518,Traci,Prescott,...,49.0,2022-01-16,0.53,6.0,173.0,10.0,0.0,0.027,0.194,0.167


In [33]:
all_reviews_summary = pd.merge(listings_and_reviews, complaints_grouped, on = 'unique_review_code', how="outer")

In [35]:
all_reviews_summary.shape

(77592, 35)

In [36]:
all_reviews_summary.T

,0,1,2,3,4,5,6,7,8,9,...,77582,77583,77584,77585,77586,77587,77588,77589,77590,77591
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,77573,77574,77575,77576,77577,77578,77579,77580,77581,77582
listing_id,3083,3083,3083,3083,3083,3083,3083,3083,3083,3083,...,572935312919648314,574751225069531168,577064541811601738,578590001628754073,578590001628754073,578590001628754073,578590001628754073,580422605482576536,580422605482576536,580422605482576536
date,2018-03-25,2018-05-26,2018-06-02,2018-06-18,2018-07-01,2018-07-06,2018-07-15,2018-07-20,2018-07-25,2018-08-17,...,2022-03-13,2022-03-06,2022-03-20,2022-03-12,2022-03-13,2022-03-14,2022-03-18,2022-03-16,2022-03-19,2022-03-20
reviewer_id,37009535,3983919,140584214,177791086,164288720,164288720,35802913,435381,87491007,18412256,...,448109600,217596381,266289368,66837051,2156668,37370316,54299704,208526814,389675668,14595781
reviewer_name,Johannes,O,Jerry,Elizabeth,Alen,Alen,William,Ritesh,Jeremy,Sallie,...,Elaine,Melissa,Anna,Sally,Nicole,Francine,Kelsey,Nazarea,Cristian,Leyla
comments,"Realy good opportunity for a cheap stay in San Francisco close to the City Center (approx. 20 min to Powel Street station). Guesthouse is nice and cosy and has an amazing garden. The neighbourhood might feel a little scary after dark, but to be fair I can not really compare it to other places in San Francisco and we never had any problems.","Loved the vibe in the house. Great location. I was there for only a very brief stay during a trip for an academic conference just across the bay in the Embarcadero, so this spot was very convenient. Great hospitality.",great hospitality and very clean,"Great place in a very central location! Her place is clean, there are 4 other rooms for rent, but there are locks on every personal room. She also does have cleaning people come three times a week which is great. Never had an issue with parking, shower gets food and hot and the space is very cute! Great value for the money you’re paying. And Traci, the host is the best! Very helpful, nice, and welcoming. Never felt unsafe in the neighborhood, for the most part it’s pretty quiet and people keep to themselves. She’s been renting out for ten years, and it shows. Very well run. Cheers!",Beautiful patio and quiet neighborhood. Nice host! Highly recommended!,"Place was so good, that we had to extend our stay! Amazing host, beautiful garden for hanging out and relaxing. Still highly recommended!","If you are visiting San Fran for work or for pleasure, you will pay a lot for housing. A lot. And maybe you want the downtown experience. But I don't want to pay a lot for housing, and I don't want the dirty, noisy, dangerous streets of San Fran. Oakland is a 10 minute Bart ride away, and the West Oakland stop is as close as you can get to San Fran without being in San Fran. So what's left? Well, you want to be close to the BART. Bingo, literally 5 min walk. Now you need a nice room in a nice clean house, and a real nice hostess in Traci (great conversation!). You're all set. This place is perfect for me.","Great place, 5 min walk to BART and barely 10 mins to SF. Quick checkin and Traci and her staff were always accessible to help out and answer questions.",Traci was a wonderful and caring host. She was flexible and understanding with arrival. The building is a few blocks walking from the West Oakland BART station which made commuting to SF a breeze.,"This is a funky, friendly place. Good experience.",...,"Beautiful house, a lot of stairs to get to the room. Beautiful view. Very sweet host.","This place is filthy. It’s a shame because it has a beautiful view from the deck but you can tell they do not clean this place. Terrible communication too. I wasn’t given check in instructions when I booked. I called to ask what they were the morning I was set to check in, since he wouldn’t respond to my Airbnb messages and he didn’t even know I booked it and said I couldn’t come because it was too early, yet the app told me check in was from 8am-10am and

In [38]:
all_reviews_summary['cat_1'] = all_reviews_summary['cat_1'].fillna(0)
all_reviews_summary['cat_2'] = all_reviews_summary['cat_2'].fillna(0)
all_reviews_summary['cat_3'] = all_reviews_summary['cat_3'].fillna(0)
all_reviews_summary['cat_4'] = all_reviews_summary['cat_4'].fillna(0)
all_reviews_summary['cat_5'] = all_reviews_summary['cat_5'].fillna(0)
all_reviews_summary['cat_6'] = all_reviews_summary['cat_6'].fillna(0)
all_reviews_summary['cat_7'] = all_reviews_summary['cat_7'].fillna(0)
all_reviews_summary['cat_8'] = all_reviews_summary['cat_8'].fillna(0)
all_reviews_summary['cat_9'] = all_reviews_summary['cat_9'].fillna(0)
all_reviews_summary['cat_10'] = all_reviews_summary['cat_10'].fillna(0)

In [40]:
all_reviews_summary.T

,0,1,2,3,4,5,6,7,8,9,...,77582,77583,77584,77585,77586,77587,77588,77589,77590,77591
Unnamed: 0,0,1,2,3,4,5,6,7,8,9,...,77573,77574,77575,77576,77577,77578,77579,77580,77581,77582
listing_id,3083,3083,3083,3083,3083,3083,3083,3083,3083,3083,...,572935312919648314,574751225069531168,577064541811601738,578590001628754073,578590001628754073,578590001628754073,578590001628754073,580422605482576536,580422605482576536,580422605482576536
date,2018-03-25,2018-05-26,2018-06-02,2018-06-18,2018-07-01,2018-07-06,2018-07-15,2018-07-20,2018-07-25,2018-08-17,...,2022-03-13,2022-03-06,2022-03-20,2022-03-12,2022-03-13,2022-03-14,2022-03-18,2022-03-16,2022-03-19,2022-03-20
reviewer_id,37009535,3983919,140584214,177791086,164288720,164288720,35802913,435381,87491007,18412256,...,448109600,217596381,266289368,66837051,2156668,37370316,54299704,208526814,389675668,14595781
reviewer_name,Johannes,O,Jerry,Elizabeth,Alen,Alen,William,Ritesh,Jeremy,Sallie,...,Elaine,Melissa,Anna,Sally,Nicole,Francine,Kelsey,Nazarea,Cristian,Leyla
comments,"Realy good opportunity for a cheap stay in San Francisco close to the City Center (approx. 20 min to Powel Street station). Guesthouse is nice and cosy and has an amazing garden. The neighbourhood might feel a little scary after dark, but to be fair I can not really compare it to other places in San Francisco and we never had any problems.","Loved the vibe in the house. Great location. I was there for only a very brief stay during a trip for an academic conference just across the bay in the Embarcadero, so this spot was very convenient. Great hospitality.",great hospitality and very clean,"Great place in a very central location! Her place is clean, there are 4 other rooms for rent, but there are locks on every personal room. She also does have cleaning people come three times a week which is great. Never had an issue with parking, shower gets food and hot and the space is very cute! Great value for the money you’re paying. And Traci, the host is the best! Very helpful, nice, and welcoming. Never felt unsafe in the neighborhood, for the most part it’s pretty quiet and people keep to themselves. She’s been renting out for ten years, and it shows. Very well run. Cheers!",Beautiful patio and quiet neighborhood. Nice host! Highly recommended!,"Place was so good, that we had to extend our stay! Amazing host, beautiful garden for hanging out and relaxing. Still highly recommended!","If you are visiting San Fran for work or for pleasure, you will pay a lot for housing. A lot. And maybe you want the downtown experience. But I don't want to pay a lot for housing, and I don't want the dirty, noisy, dangerous streets of San Fran. Oakland is a 10 minute Bart ride away, and the West Oakland stop is as close as you can get to San Fran without being in San Fran. So what's left? Well, you want to be close to the BART. Bingo, literally 5 min walk. Now you need a nice room in a nice clean house, and a real nice hostess in Traci (great conversation!). You're all set. This place is perfect for me.","Great place, 5 min walk to BART and barely 10 mins to SF. Quick checkin and Traci and her staff were always accessible to help out and answer questions.",Traci was a wonderful and caring host. She was flexible and understanding with arrival. The building is a few blocks walking from the West Oakland BART station which made commuting to SF a breeze.,"This is a funky, friendly place. Good experience.",...,"Beautiful house, a lot of stairs to get to the room. Beautiful view. Very sweet host.","This place is filthy. It’s a shame because it has a beautiful view from the deck but you can tell they do not clean this place. Terrible communication too. I wasn’t given check in instructions when I booked. I called to ask what they were the morning I was set to check in, since he wouldn’t respond to my Airbnb messages and he didn’t even know I booked it and said I couldn’t come because it was too early, yet the app told me check in was from 8am-10am and

In [41]:
airbnb_issues_grouped = all_reviews_summary.groupby('listing_id').agg(
             cat_1 = ('cat_1', 'sum'),
             cat_2 = ('cat_2', 'sum'),
             cat_3 = ('cat_3', 'sum'),
             cat_4 = ('cat_4', 'sum'),
             cat_5 = ('cat_5', 'sum'),
             cat_6 = ('cat_1', 'sum'),
             cat_7 = ('cat_1', 'sum'),
             cat_8 = ('cat_1', 'sum'),
             cat_9 = ('cat_1', 'sum'),
             cat_10 = ('cat_1', 'sum'))

airbnb_issues_grouped.reset_index()  

,listing_id,cat_1,cat_2,cat_3,cat_4,cat_5,cat_6,cat_7,cat_8,cat_9,cat_10
0,3083,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,5739,1.0,0.0,5.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
2,23637,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,24916,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,30026,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
1800,39646877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1801,39670902,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1802,39759036,2.0,1.0,2.0,0.0,0.0,2.0,2.0,2.0,2.0,2.0
1803,39782053,3.0,0.0,0.0,0.0,0.0,3.0,3.0,3.0,3.0,3.0


# Exporting the Data for Visualization in Tableau

In [43]:
# all negative reviews with categories included
negative_reviews_summary.to_csv('negative_reviews_summary.csv', sep ='\t')
files.download("negative_reviews_summary.csv")

## all reviews with categories included, lots of blanks/ 0s
all_reviews_summary.to_csv('all_reviews_summary.csv', sep ='\t')
files.download("all_reviews_summary.csv")

## categories grouped by airbnb
airbnb_issues_grouped.to_csv('airbnb_issues_grouped.csv', sep ='\t')
files.download("airbnb_issues_grouped.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>